In [13]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [15]:
# import sys
# print(sys.executable)
# print(sys.version)
# print(sys.version_info)

In [17]:
with open('token.pickle', 'rb') as token:
    creds = pickle.load(token)

In [16]:
# flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
# creds = flow.run_local_server(port=0)

In [18]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']
creds = None

# # If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('drive', 'v3', credentials=creds)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=378815117574-d1326urt3g82g3fc0cn68ecoooe03uhg.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A64689%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.metadata.readonly&state=AbJWoqDgliK0FelwyrCj4s8AUGp0Vw&access_type=offline


In [19]:
q = [
    {
        "id": "1epV5S829Ae3pTz2R8PB-zSBoWmfZOOmk",
        "path": "year-in-review-2024/"
    }
]
# with open("img_folders.csv") as fin:
#     for line in fin:
#         line = line.strip().split(",")
#         q.append({
#             "id": line[1],
#             "path": line[0]
#         })


fin = open("_imgs.csv", "w")
while len(q)>0:
    target_folder = q.pop()
    target_folder_path = target_folder["path"]
    target_folder_id = target_folder["id"]
    print("ls", target_folder_path)

    page_token = None
    while True:
        response = service.files().list(q=f"('{target_folder_id}' in parents) and (mimeType contains 'image/' or mimeType contains 'video/')",
                                            spaces='drive',
                                            fields='nextPageToken, files(id, name)',
                                            pageToken=page_token).execute()
        for file in response.get('files', []):
            # Process change
            o = {
                "id": file.get('id'),
                "path": target_folder_path+file.get('name')
            }

            fin.write(f"{o['path']},{o['id']}\n")
        page_token = response.get('nextPageToken', None)
        if page_token is None:
            break

fin.close()

ls year-in-review-2024/
